### Load tensorflow

In [0]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
tf.enable_eager_execution(
    config=None,
    device_policy=None,
    execution_mode=None
)


### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('prices.csv')

In [8]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [6]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(['date','symbol'], axis = 1)

In [14]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data1 = data[:1000]

In [16]:
data1.tail()

,open,close,low,high,volume
995,63.310001,63.590000,63.240002,63.639999,2133200.0
996,27.160000,26.990000,26.680000,27.299999,1982400.0
997,28.320000,28.770000,28.010000,28.809999,37152800.0
998,44.000000,44.799999,43.750000,44.810001,6568600.0
999,36.080002,37.139999,36.009998,37.230000,5604300.0


### Convert Float64 to Float32

In [0]:
data1 = data1.astype('float32')

In [18]:
data1.dtypes

open      float32
close     float32
low       float32
high      float32
volume    float32
dtype: object

### Divide the data into train and test sets

In [0]:
X = data1.drop('close', axis = 1)
Y = data1['close']

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
X_train_transform = scaler.fit_transform(X_train) 
X_test_transform = scaler.fit_transform(X_test) 

In [0]:
X_train_transform = X_train_transform.astype('float32')
X_test_transform = X_test_transform.astype('float32')

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [28]:
type(X_train)

pandas.core.frame.DataFrame

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y


4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [48]:
import numpy as np
#Train for 100 Steps
for i in range(100):
    
    w, b = train(X_train_transform, np.array(y_train), w, b, learning_rate=0.01)
   
    print('Current Loss on iteration', i, 
          loss(np.array(y_train), prediction(X_train_transform, w, b)).numpy())

Current Loss on iteration 0 6849.1904
Current Loss on iteration 1 6686.1616
Current Loss on iteration 2 6529.797
Current Loss on iteration 3 6379.8247
Current Loss on iteration 4 6235.9854
Current Loss on iteration 5 6098.0254
Current Loss on iteration 6 5965.7065
Current Loss on iteration 7 5838.7974
Current Loss on iteration 8 5717.076
Current Loss on iteration 9 5600.332
Current Loss on iteration 10 5488.36
Current Loss on iteration 11 5380.9663
Current Loss on iteration 12 5277.9634
Current Loss on iteration 13 5179.1714
Current Loss on iteration 14 5084.4185
Current Loss on iteration 15 4993.539
Current Loss on iteration 16 4906.3755
Current Loss on iteration 17 4822.7754
Current Loss on iteration 18 4742.593
Current Loss on iteration 19 4665.689
Current Loss on iteration 20 4591.9287
Current Loss on iteration 21 4521.184
Current Loss on iteration 22 4453.3315
Current Loss on iteration 23 4388.2534
Current Loss on iteration 24 4325.8354
Current Loss on iteration 25 4265.9697
Curre

### Get the shapes and values of W and b

In [51]:
print(w)
print(w.shape)

tf.Tensor(
[[5.5900035]
 [5.559609 ]
 [5.6214337]
 [1.2809763]], shape=(4, 1), dtype=float32)
(4, 1)


In [52]:
print(b)
print(b.shape)

tf.Tensor([56.073418], shape=(1,), dtype=float32)
(1,)


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
model = tf.keras.Sequential()


In [0]:
model.add(tf.keras.layers.Dense(1,input_shape = (4,)))

In [0]:
model.compile(optimizer='sgd', loss = 'mse')

### Execute the model

In [64]:
model.fit(X_train_transform, np.array(y_train), epochs=100)

Epoch 1/100
800/800 [==============================] - 0s 40us/sample - loss: 317.4956
Epoch 2/100
800/800 [==============================] - 0s 30us/sample - loss: 310.7304
Epoch 3/100
800/800 [==============================] - 0s 30us/sample - loss: 304.2859
Epoch 4/100
800/800 [==============================] - 0s 30us/sample - loss: 297.8409
Epoch 5/100
800/800 [==============================] - 0s 30us/sample - loss: 291.4951
Epoch 6/100
800/800 [==============================] - 0s 29us/sample - loss: 285.3793
Epoch 7/100
800/800 [==============================] - 0s 30us/sample - loss: 279.5129
Epoch 8/100
800/800 [==============================] - 0s 28us/sample - loss: 273.6224
Epoch 9/100
800/800 [==============================] - 0s 30us/sample - loss: 267.9444
Epoch 10/100
800/800 [==============================] - 0s 30us/sample - loss: 262.2600
Epoch 11/100
800/800 [==============================] - 0s 30us/sample - loss: 256.6211
Epoch 12/100
800/800 [===================

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [106]:
df_iris = pd.read_csv('Iris.csv')
df_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X = df_iris.drop(columns=['Species','Id'], axis = 1)
Y = df_iris['Species']


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
Y = pd.get_dummies(Y)

### Divide the dataset into Training and test (70:30)

In [0]:
X_train, X_test, y_train , y_test = train_test_split(X,Y, test_size = 0.3)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
model_iris = tf.keras.Sequential()
model_iris.add(tf.keras.layers.BatchNormalization())

In [0]:
model_iris.add(tf.keras.layers.Dense(10,input_shape = (4,), activation='sigmoid'))

In [0]:
model_iris.add(tf.keras.layers.Dense(8, activation='sigmoid'))

In [0]:
model_iris.add(tf.keras.layers.Dense(3, activation='softmax'))

### Fitting the model and predicting 

In [0]:
model_iris.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [117]:
y_train.shape

(45, 4)

In [121]:
model_iris.fit(np.array(X_train),np.array(y_train),  epochs=100, batch_size = 12)

Epoch 1/100
105/105 [==============================] - 0s 887us/sample - loss: 1.0808 - acc: 0.4095
Epoch 2/100
105/105 [==============================] - 0s 114us/sample - loss: 1.0795 - acc: 0.3810
Epoch 3/100
105/105 [==============================] - 0s 119us/sample - loss: 1.0783 - acc: 0.4095
Epoch 4/100
105/105 [==============================] - 0s 143us/sample - loss: 1.0776 - acc: 0.4095
Epoch 5/100
105/105 [==============================] - 0s 139us/sample - loss: 1.0767 - acc: 0.4095
Epoch 6/100
105/105 [==============================] - 0s 114us/sample - loss: 1.0755 - acc: 0.3810
Epoch 7/100
105/105 [==============================] - 0s 109us/sample - loss: 1.0757 - acc: 0.4190
Epoch 8/100
105/105 [==============================] - 0s 108us/sample - loss: 1.0747 - acc: 0.4095
Epoch 9/100
105/105 [==============================] - 0s 111us/sample - loss: 1.0726 - acc: 0.4667
Epoch 10/100
105/105 [==============================] - 0s 133us/sample - loss: 1.0726 - acc: 0.3905

### Report Accuracy of the predicted values

In [122]:
model_iris.evaluate(X_test, y_test)

45/45 [==============================] - 0s 2ms/sample - loss: 0.9420 - acc: 0.9333


[0.9419904324743483, 0.93333334]